In [1]:
import os

In [2]:
prev=os.getcwd()

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\Muhammad Hasnain\\Desktop\\KidneyDiseaseClassification'

In [ ]:
import dagshub
# https://dagshub.com/hasnainalamglt143/Kidney-Disease-classification.mlflow
dagshub.init(repo_owner='hasnainalamglt143', repo_name='Kidney-Disease-classification', mlflow=True)

import mlflow
with mlflow.start_run():
  mlflow.log_param('parameter name', 'value')
  mlflow.log_metric('metric name', 1)

In [19]:
from pathlib import Path
from dataclasses import dataclass
from cnn_classifier.constants import PARAMS_FILE_PATH,CONFIG_FILE_PATH
from cnn_classifier.utils.common import read_yaml,create_directories
import tensorflow as tf

In [9]:
@dataclass(frozen=True)
class ModelEvaluationEntity:
    training_data_path:Path
    trained_model_path:Path
    all_params:dict
    

In [17]:
class ModelEvaluationConfigManager:
    def __init__(self):
        self.config=read_yaml(CONFIG_FILE_PATH)
        self.params=read_yaml(PARAMS_FILE_PATH)
        create_directories([self.config.artifacts_root])

    def get_eval_config(self):
        return ModelEvaluationEntity(training_data_path=Path(os.path.join(self.config.data_ingestion.root_dir,"kidney_images")),trained_model_path=Path(self.config.model_training.trained_model_path),all_params=self.params)


            

In [18]:
obj=ModelEvaluationConfigManager()
obj.get_eval_config()

[2025-09-30 23:40:14,147] : common: INFO: yaml file 'config\config.yaml' loaded 
[2025-09-30 23:40:14,151] : common: INFO: yaml file 'params.yaml' loaded 
[2025-09-30 23:40:14,151] : common: INFO: created directory at: artifacts


ModelEvaluationEntity(training_data_path=WindowsPath('artifacts/data_ingestion/kidney_images'), trained_model_path=WindowsPath('artifacts/trained_model/trained_model.keras'), all_params=ConfigBox({'AUGMENTATION': True, 'INCLUDE_TOP': False, 'EPOCHS': 1, 'CLASSES': 2, 'BATCH_SIZE': 16, 'IMAGE_SHAPE': [224, 224, 3], 'LEARNING_RATE': 0.01, 'WEIGHTS': 'imagenet'}))

In [26]:
class ModelEvaluation:
    def __init__(self,config:ModelEvaluationConfigManager):
        self.config=config.get_eval_config()
    
    
    def get_trained_model(self):
        model=tf.keras.models.load_model(self.config.trained_model_path)
        return model

    def _valid_generator(self):
        datagenerator_kwargs = dict( rescale = 1./255, validation_split=0.30 ) 
        dataflow_kwargs = dict( target_size=self.config.all_params.IMAGE_SHAPE[:-1], batch_size=self.config.all_params.BATCH_SIZE, interpolation="bilinear" ) 
        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
              **datagenerator_kwargs )
         
        self.valid_generator = valid_datagenerator.flow_from_directory( 
             directory=self.config.training_data_path, 
             subset="validation", 
             shuffle=False, **dataflow_kwargs ) 
        
    def model_evaluation(self):
        self.model=self.get_trained_model()
        self._valid_generator()
        print(self.model)




In [27]:
obj2=ModelEvaluation(obj)
obj2.model_evaluation()

Found 139 images belonging to 2 classes.
<Sequential name=sequential, built=True>
